In [5]:
!pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=ebbd1f60a4bba8a58edd9dad4c1ebbb63316dfeb21ae02d81fe033842932eb9b
  Stored in directory: c:\users\ehdrn\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor


In [3]:
!pip install keras

In [1]:
!pip install konlpy

  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.1
    Uninstalling beautifulsoup4-4.9.1:
      Successfully uninstalled beautifulsoup4-4.9.1


In [6]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.
<br>
<br>
<br>

In [3]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
wear_data = pd.read_csv("/content/drive/My Drive/wear.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.

(15826, 20)
(8381,) (7445,)


In [6]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))
print(store_arr[-1])
print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)

7301
7301
요즘 파스텔 톤이 유행이에요
요즘 유행하는 색깔이 뭐예요?


In [7]:
question = []
answer = []

for Q in customer_arr:
    question.append(Q.replace("[^\w]", " "))

for A in store_arr:
    answer.append(A.replace("[^\w]", " "))

len(question), len(answer)

<br>
<br>
# 단어 사전 생성

In [9]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [10]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 신발 은 여기 있는 게 다예 요
A : 네 성인 이나 아동 다 있어요 발 사이즈 몇 신으세요

Q : 230 이요
A : 편하게 신 을 수 있는 거 찾으세요

Q : 네 봄 이니까 편하게 신 을 수 있는 거
A : 이런 건 어떠세요 이런 거도 신발 무척 편하거든요

Q : 굽 좀 높은 거 없나요
A : 봄 상품 은 아직 어른 제품 이 많이 안 나왔습니다

Q : 언제 들어와요
A : 이번 주 지나면 들어올 거 예요

Q : 이 거 는 가죽 이에요
A : 가죽 아니고 쎄 무예 요

Q : 가죽 은 얼마 예요
A : 2만 9천 원 입니다

Q : 털 달린 거 저 거 는 사이즈 있어요
A : 230 이 없어요 이 거 한 번 신어 보세요

Q : 좀 크네 또 안 들어와요
A : 네 이건 다 끝났어요

Q : 가방 매는 거 보고 있어요
A : 여기 있어요



In [13]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [15]:
print("손님과 점원의 말에서 사용된 총 단어의 수 :",len(words))

손님과 점원의 말에서 사용된 총 단어의 수 : 6403


In [16]:
# 단어 출력
words[:20]


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '나왔나요',
 '있죠',
 '적합한게',
 '금지',
 '가능해요',
 '된건데',
 '국제',
 '있어서요',
 '받을수있는건',
 '있다가',
 '되겠다',
 '파손',
 '아프나요',
 '파커',
 '나왔네요',
 '팔꿈치']

In [17]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [19]:
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '다니기': 4,
 '그렇치요': 5,
 '나오긴': 6,
 '빠지면': 7,
 '솜': 8,
 '비슷하지': 9,
 '골고루': 10,
 '롱패딩': 11,
 '신다': 12,
 '튀는것': 13,
 '우리나라': 14,
 '클까': 15,
 '다른데요': 16,
 '둘러보겠습니다': 17,
 '행사': 18,
 '20000원': 19,
 '무역': 20,
 '브로': 21,
 '어둡나요': 22,
 '겨자': 23,
 '가요': 24,
 '않나요': 25,
 '화': 26,
 '하는데': 27,
 '무늬': 28,
 '하더라': 29,
 '봐야': 30,
 '따': 31,
 '두었습니다': 32,
 '껴': 33,
 '받이': 34,
 '맞추시려면은': 35,
 '도리': 36,
 '편': 37,
 '폭': 38,
 '나오는데': 39,
 '가위': 40,
 '잘나가거든요': 41,
 '입어주세요': 42,
 '닫으세요': 43,
 '개인': 44,
 '나서': 45,
 '창': 46,
 '나오는': 47,
 '심한데': 48,
 '곳': 49,
 '다니는데': 50,
 '어떡해요': 51,
 '조명': 52,
 '이왕이면': 53,
 '나오지요': 54,
 '있더라고요': 55,
 '옥': 56,
 '사십이': 57,
 '감사해요': 58,
 '낫긴': 59,
 '에다가': 60,
 '입기': 61,
 '어울리시네요': 62,
 '아이고': 63,
 '찍이가': 64,
 '신으려면': 65,
 '가능하시고요': 66,
 'SS': 67,
 '98000원': 68,
 '큰': 69,
 '공기': 70,
 '흰': 71,
 '밑창': 72,
 '약간': 73,
 '하셨는데요': 74,
 '전자': 75,
 '들어가야겠다': 76,
 '반대편': 77,
 '일제': 78,
 '2만원': 79,
 '지워주세요': 80,
 '새거로': 81,
 '녹슬지': 8

In [18]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [19]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (신발 은 여기 있는 게 다예 요)
print(question[0])
x_encoder[0]


신발 은 여기 있는 게 다예 요


array([ 701, 5039, 2081,  624, 2131, 1000, 4077,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [20]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (<START> 신발 은 여기 있는 게 다예 요)
print(question[0])
x_decoder[0]


신발 은 여기 있는 게 다예 요


array([   1, 3245, 4807, 4067, 4514, 1695, 2416, 1888, 1754, 4427, 2946,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [21]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 입력 출력 (신발 은 여기 있는 게 다예 요 <END>)
print(question[0])
y_decoder[0]


신발 은 여기 있는 게 다예 요


array([3245, 4807, 4067, 4514, 1695, 2416, 1888, 1754, 4427, 2946,    2,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [22]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

<br>
<br>
# 모델 생성

In [23]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

In [24]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

<br>
<br>
# 훈련 및 테스트

In [28]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [26]:
x_encoder[2].reshape(1, x_encoder[2].shape[0])

array([[3245, 1886, 6060, 4767, 1496, 5142, 6206,  624, 3873,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [29]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=1,
                        batch_size=64,
                        verbose=1)
    # '네 봄 이니까 편하게 신 을 수 있는 거'에 대한 대답
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
115/115 [==============================] - 4s 38ms/step - loss: 1.2817 - accuracy: 0.8150
네 이 은 

Total Epoch : 2
115/115 [==============================] - 4s 38ms/step - loss: 1.2170 - accuracy: 0.8186
네 이 이 

Total Epoch : 3
115/115 [==============================] - 5s 39ms/step - loss: 1.1802 - accuracy: 0.8212
네 은 이 

Total Epoch : 4
115/115 [==============================] - 5s 40ms/step - loss: 1.1474 - accuracy: 0.8248
네 거 거 

Total Epoch : 5
115/115 [==============================] - 5s 39ms/step - loss: 1.1163 - accuracy: 0.8284
네 옷 이 

Total Epoch : 6
115/115 [==============================] - 4s 39ms/step - loss: 1.0841 - accuracy: 0.8329
네 거 어떻게 

Total Epoch : 7
115/115 [==============================] - 5s 40ms/step - loss: 1.0540 - accuracy: 0.8368
네 거 어떻게 

Total Epoch : 8
115/115 [==============================] - 4s 38ms/step - loss: 1.0256 - accuracy: 0.8398
네 제품 어떻게 

Total Epoch : 9
115/115 [==============================] - 4s 39ms/step - loss: 1

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

<br>
<br>
# 문장 생성

In [32]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [33]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [34]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('이 티셔츠 사이즈 있나요')
input_seq


array([[1541,  307, 1754, 1072,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [35]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'네 잠시 만 기다려주세요 '

데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [36]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('교환 가능한가요?')
input_seq


array([[2393, 5592,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [37]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'네 가능합니다 '

In [43]:
# for train data predict
for seq_index in range(0,100):

  print("고객 : ",question[seq_index])
  print("정답점원 :",answer[seq_index])
  print("AI점원 :",generate_text(make_predict_input(question[seq_index])))
  print("\n")

고객 :  신발 은 여기 있는 게 다예 요
정답점원 : 네 성인 이나 아동 다 있어요 발 사이즈 몇 신으세요
AI점원 : 네 


고객 :  230 이요
정답점원 : 편하게 신 을 수 있는 거 찾으세요
AI점원 : 네 그럼요 


고객 :  네 봄 이니까 편하게 신 을 수 있는 거
정답점원 : 이런 건 어떠세요 이런 거도 신발 무척 편하거든요
AI점원 : 네 


고객 :  굽 좀 높은 거 없나요
정답점원 : 봄 상품 은 아직 어른 제품 이 많이 안 나왔습니다
AI점원 : 네 이 쪽 에 있습니다 


고객 :  언제 들어와요
정답점원 : 이번 주 지나면 들어올 거 예요
AI점원 : 네 


고객 :  이 거 는 가죽 이에요
정답점원 : 가죽 아니고 쎄 무예 요
AI점원 : 네 


고객 :  가죽 은 얼마 예요
정답점원 : 2만 9천 원 입니다
AI점원 : 그 옷 은 이만 원 이에요 


고객 :  털 달린 거 저 거 는 사이즈 있어요
정답점원 : 230 이 없어요 이 거 한 번 신어 보세요
AI점원 : 네 


고객 :  좀 크네 또 안 들어와요
정답점원 : 네 이건 다 끝났어요
AI점원 : 네 


고객 :  가방 매는 거 보고 있어요
정답점원 : 여기 있어요
AI점원 : 네 


고객 :  가격 이 얼마 예요
정답점원 : 이 종류 는 2만 원 이고 이 종류 는 3만 8천 원 이에요
AI점원 : 이 거 는 삼만 원 이에요 


고객 :  가죽 으로 된 거 는 없어요
정답점원 : 가죽 은 없고 레자 만 있어요
AI점원 : 네 


고객 :  레자 는 얼마 예요
정답점원 : 5만 5천 원요
AI점원 : 그 가방 은 이만 원 이에요 


고객 :  이 거 는 천이 죠
정답점원 : 네 맞아요
AI점원 : 네 


고객 :  이건 얼마 예요
정답점원 : 그것 도 5만 5천 원요
AI점원 : 그 가방 은 이만 원 이에요 


고객 :  이 거 끈 은 따로 없어요
정답점원 : 안 에 있어요
AI점원 : 네 


고객 :  내일 은 문 열어요
정답점원 : 휴무 입니다
A